In [1]:
import json
import csv
from tqdm import tqdm

# Load objects.json
print("🔍 Đang tải objects.json...")
with open("E:/Download/VG/objects.json", "r") as f:
    objects_data = json.load(f)

# Tạo ánh xạ: image_id + object_id → synset (nếu có)
print("📌 Đang tạo ánh xạ object_id → synset/name...")
object_map = {}  # key = (image_id, object_id), value = synset hoặc name

for item in tqdm(objects_data):
    image_id = item["image_id"]
    for obj in item["objects"]:
        obj_id = obj["object_id"]
        names = obj["names"]
        name = names[0].strip().lower() if names else None
        synsets = obj["synsets"]
        key = (image_id, obj_id)
        if synsets and len(synsets) > 0:
            object_map[key] = synsets[0]  # dùng synset đầu tiên
        else:
            object_map[key] = name  # fallback dùng name

# Load relationships.json
print("🔍 Đang tải relationships.json...")
with open("E:/Download/VG/relationships.json", "r") as f:
    relationships_data = json.load(f)

triplets = []

print("🔄 Đang kết hợp triplet với synset từ objects.json...")
for item in tqdm(relationships_data):
    image_id = item["image_id"]
    for rel in item.get("relationships", []):
        try:
            subj_id = rel["subject"]["object_id"]
            obj_id = rel["object"]["object_id"]
            predicate = rel["predicate"].strip().lower()

            subject = object_map.get((image_id, subj_id), rel["subject"]["name"].strip().lower())
            obj = object_map.get((image_id, obj_id), rel["object"]["name"].strip().lower())

            triplets.append((subject, predicate, obj, image_id))
        except Exception as e:
            continue  # Skip nếu thiếu thông tin

# Ghi ra file CSV
output_file = "f_vg_triplets.csv"
print(f"💾 Ghi {len(triplets)} triplet vào {output_file}...")
with open(output_file, "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "predicate", "object", "image_id"])
    writer.writerows(triplets)

print("✅ Hoàn tất!")


🔍 Đang tải objects.json...
📌 Đang tạo ánh xạ object_id → synset/name...


100%|███████████████████████████████████████████████████████████████████████| 108077/108077 [00:03<00:00, 29584.79it/s]


🔍 Đang tải relationships.json...
🔄 Đang kết hợp triplet với synset từ objects.json...


100%|████████████████████████████████████████████████████████████████████████| 108077/108077 [00:53<00:00, 2036.38it/s]


💾 Ghi 1737904 triplet vào f_vg_triplets.csv...
✅ Hoàn tất!
